# Práctico 1

[Enunciado](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/Practico.md) del trabajo práctico.

**Implementación de red neuronal [Perceptrón Multicapa](https://en.wikipedia.org/wiki/Multilayer_perceptron) (MLP).**

## Integrantes
- Mauricio Caggia
- Luciano Monforte
- Gustavo Venchiarutti
- Guillermo Robiglio

## Importaciones

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

## Constantes

In [2]:
ARCHIVO_SET_DE_ENTRENAMIENTO = './data/meli-challenge-2019/spanish.train.jsonl.gz'
ARCHIVO_DE_ENTRENAMIENTO_REDUCIDO = './data/train.json'
ARCHIVO_SET_DE_PRUEBA = './data/meli-challenge-2019/spanish.test.jsonl.gz'
ARCHIVO_DE_PRUEBA_REDUCIDO = './data/test.json'
ARCHIVO_SET_DE_VALIDACION = './data/meli-challenge-2019/spanish.validation.jsonl.gz'
ARCHIVO_DE_VALIDACION_REDUCIDO = './data/validation.json'
ARCHIVO_TOKENS = './data/meli-challenge-2019/spanish_token_to_index.json.gz'

## Carga de datos

Completa

In [ ]:
%%time
file_paths = [ARCHIVO_SET_DE_ENTRENAMIENTO, # Ingresar opción 0 👁 ⚠ Tarda más de 2 minutos en cargar y puede que haya un desbordamiento de RAM o muera el kernel⚠
              ARCHIVO_SET_DE_PRUEBA,  # Ingresar opción 1
              ARCHIVO_SET_DE_VALIDACION] # Ingresar opción 2 ⚠ Tarda más de 30 segundos en cargar
i = int(input('Ingresar opción para carga de archivo (0 a 2): '))
df = pd.read_json(path_or_buf=file_paths[i], lines=True)

Reducida

In [3]:
%%time
file_paths = [ARCHIVO_DE_ENTRENAMIENTO_REDUCIDO, # Ingresar opción 0
              ARCHIVO_DE_PRUEBA_REDUCIDO, # Ingresar opción 1
              ARCHIVO_DE_VALIDACION_REDUCIDO] # Ingresar opción 2
i = int(input('Ingresar opción para carga de archivo (0 a 2): '))
df_reducido = pd.read_json(path_or_buf=file_paths[i], lines=False)

Ingresar opción para carga de archivo (0 a 2): 2
CPU times: user 3.86 s, sys: 508 ms, total: 4.37 s
Wall time: 6.37 s


In [ ]:
tokens = pd.read_json(path_or_buf=ARCHIVO_TOKENS, lines=True).T

## Análisis y visualización de los datos

El **set de entrenamiento** original tiene 4895280 registros con valores no nulos y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Se dispone de 4508043 registros no confiables y 387237 registros confiables.
- **title**: El título que se asignó al producto.
- **category**: La categoría que se asignó al producto.
- **split**: El tipo de dataset. _train_ para el set de entrenamiento.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado. Esta información es la que se utilizará para entrenar el modelo.
- **target**: El número que corresponde a cada categoría.
- **n_labels**: Cantidad de etiquetas numéricas correspondientes a las distintas categorías. 632 etiquetas (0 a 631) para el caso del set de entrenamiento.
- **size**: La cantidad de registros. 4895280 registros para el caso del set de entrenamiento.

El **set de prueba** original tiene 63680 registros con valores no nulos y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Todas las etiquetas de este dataset son confiables.
- **title**: El título que se asignó al producto.
- **category**: La categoría que se asignó al producto.
- **split**: El tipo de dataset. _test_ para el set de prueba.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado. Esta información es la que se utilizará para entrenar el modelo.
- **target**: El número que corresponde a cada categoría.
- **n_labels**: Cantidad de etiquetas numéricas correspondientes a las distintas categorías. 632 etiquetas (0 a 631) para el caso del set de prueba.
- **size**: La cantidad de registros. 63680 registros para el caso del set de prueba.

El **set de validación** original tiene 1223820 registros con valores no nulos y 10 columnas. Las columnas de dicho dataset son:
- **language**: El idioma del dataset (españor o portugués). En el trabajo práctico utilizaremos solamente el dataset es español.
- **label_quality**: Calidad de la etiqueta (confiable o no confiable). Se dispone de 1127189 registros no confiables y 96631 registros confiables.
- **title**: El título que se asignó al producto.
- **category**: La categoría que se asignó al producto.
- **split**: El tipo de dataset. _validation_ para el set de prueba.
- **tokenized_title**: El título tokenizado. Esto significa que los datos fueron preprocesados.
- **data**: El número asignado a cada palabra del título tokenizado. Esta información es la que se utilizará para entrenar el modelo.
- **target**: El número que corresponde a cada categoría.
- **n_labels**: Cantidad de etiquetas numéricas correspondientes a las distintas categorías. 632 etiquetas (0 a 631) para el caso del set de validación.
- **size**: La cantidad de registros. 1223820 registros para el caso del set de validación.

El archivo **spanish_token_to_index** tiene las 50002 correspondencias que existen entre las palabras tokenizadas del título y las etiquetas numéricas bajo la columna data en los sets de entrenamiento, prueba y validación.

In [ ]:
df.head()

La siguiente celda se utiliza para reducir el dataset a partir del dataset completo.

Un atajo es cargar directamente el dataset reducido.

In [ ]:
df_reducido = df.drop(columns=['language', 'label_quality', 'title',
                               'category', 'split', 'tokenized_title',
                               'n_labels', 'size'])

In [4]:
df_reducido.head()

,data,target
0,"[457, 1480, 3450]",88
1,"[3119, 892, 1, 767, 1, 9337]",174
2,"[700, 990, 2057, 3990, 3670, 1737, 1153, 6568]",313
3,"[2722, 4404, 1406, 4405]",427
4,"[1, 3365, 7502, 10919, 849]",194


Las siguientes tres celdas se utilizan una única vez para guardar un dataset reducido para cada conjunto de datos.

In [ ]:
# df_reducido.to_json(ARCHIVO_DE_ENTRENAMIENTO_REDUCIDO)

In [ ]:
# df_reducido.to_json(ARCHIVO_DE_PRUEBA_REDUCIDO)

In [ ]:
# df_reducido.to_json(ARCHIVO_DE_VALIDACION_REDUCIDO)

### La columna *data*

El **dataset de entrenamiento** posee un total de 26129139 etiquetas (que se corresponden con una palabra toquenizada) comprendidad entre el 1 y el 50001, esto es, 50001 etiquetas diferentes que permiten anticipar que el tensor que se utilizará para el entrenamiento de la red neuronal tendrá una dimensión de 4895280 x 50001.

El **dataset de prueba** posee un total de 354208 etiquetas (que se corresponden con una palabra toquenizada) comprendidas entre el 1 y el 50001. En este conjunto de datos hay 25741 etiquetas diferentes. El tensor tendrá una dimensión de 63680 x 50001.

El **dataset de validación** posee un total de 6531439 etiquetas (que se corresponden con una palabra toquenizada) comprendidas entre el 1 y el 50001. En este conjunto de datos hay 49961 etiquetas diferentes. El tensor tendrá una dimensión de 1223820 x 50001.

**NOTA**: En ninguno de los casos aparece la etiqueta 0, por eso aquí hay un elemento menos que en la lista de las etiquetas de palabras tokenizadas.

In [5]:
data = df_reducido.data

lista_num_palabras = []
for id_palabras in data:
    lista_num_palabras += id_palabras
    

print('Cantidad de etiquetas:', len(lista_num_palabras))
print('Cantidad de etiquetas diferentes:', len(set(lista_num_palabras)))
print('Valor máximo:', max(lista_num_palabras))
print('Valor mínimo:', min(lista_num_palabras))

Cantidad de id de palabras: 6531439
Cantidad de id de palabras diferentes: 49961
Valor máximo: 50001
Valor mínimo: 1


### La columna *target*

Contiene un número entero correspondiente a cada categoría. 632 etiquetas entre 0 y 631.

In [6]:
labels = df_reducido.target.unique()
print(f'{labels.shape[0]} etiquetas ordenadas como se muestra a continuación:')
np.sort(labels)

632 etiquetas ordenadas como se muestra a continuación:


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

### Tokens y sus etiquetas

Las siguientes 3 celdas de código demuestran que la relación entre los datos bajo las columnas `tokenized_title` y `data` está dada en el archivo `spanish_token_to_index` que vincula cada palabra a un índice numérico entero.

In [ ]:
i = 0 # Un índice cualquiera para extraer datos
items = df.at[i, 'tokenized_title']
items

In [ ]:
# Comparar la salida de esta celda con la de la siguiente
df.at[i, 'data']

In [ ]:
nro_items = []
for item in items:
    id_item = tokens.loc[item][0]
    nro_items.append(id_item)
nro_items

## Construcción del Dataset

In [10]:
filas = df_reducido.shape[0]
columnas = 50001
esqueleto = np.zeros((filas, columnas), dtype=np.int8)

In [16]:
pd.DataFrame(esqueleto, columns=range(1,columnas+1))

,1,2,3,4,5,6,7,8,9,10,...,49992,49993,49994,49995,49996,49997,49998,49999,50000,50001
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223815,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1223816,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1223817,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1223818,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Dataset de PyTorch

In [ ]:
class MeLiChallengeDataset(IterableDataset):
    def __init__(self, path, transform=None):
        self.dataset_path = path
        self.transform = transform

    def __iter__(self):
        with gzip.open(self.dataset_path, "rt") as fh:
            for l in fh:
                data = json.loads(l)
                item = {
                    "data": data['title'],
                    "target": data['category']
                }
                
                if self.transform:
                    yield self.transform(item)
                else:
                    yield item

dataset = MeLiChallengeDataset("./practico/data/meli-challenge-2019/spanish.train.jsonl.gz")
dataloader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=0)
dataiter = iter(dataloader)
print(f"Sample batch:\n{dataiter.next()}")

## Construcción del Modelo

## Optimizador

In [ ]:
for input, target in dataset:
    optimizer.zero_grad()
    output = model(input)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()

## Entrenamiento del Modelo

## Evaluación del Modelo

## Guardado de los parámetros del modelo